# Add / Process Batter Data
- Now, we need to process this data and augment our game-level data frame
- For each game, we want the trailing statistics for each player designated in the starting lineup
- Need to add the statistics for those 18 players to each game
- From that, we may want to derive "team-level" averages to simplify our feature set
- NOTE: this will complicate things when we "go into production" and try and use this model for predicting new games



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',5000)
pd.set_option('display.max_rows',5000)

In [2]:
df = pd.read_csv('df_bp7.csv')
df.shape

/Users/brianlucena/.pyenv/versions/3.9.13/envs/ds3913/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (13,14,15,19,85,87,159,206) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(96248, 619)

- #### When aggregating pitcher's past performance, we use default values to "smooth out" players with little / no history
- #### With batters, this is more complicated, as different positions have different "typical" hitting capabilities
- #### (Separate issue: should we be using training data with pitchers hitting?!?!)

In [3]:
def roll_column(df, col, winsize):
    # do the standard Pandas rolling calc
    t_col = df[col].rolling(winsize, closed='left').sum().to_numpy()
    
    # for the early columns, just do a rolling sum from the beginning
    t_col[:winsize] = np.concatenate(([0],df[col].iloc[:(winsize)].cumsum().to_numpy()[:-1]))

    return(t_col)

In [4]:
## Set up position level defaults
dd = {}
dd_p = {'batavg': .100, 'obp': .150, 'slg': .180, 'slgmod': .220, 'obs': .330, 'sobat': .3}
dd_ss_c = {'batavg': .205, 'obp': .260, 'slg': .300, 'slgmod': .320, 'obs': .540, 'sobat': .25}
dd_2b_3b = {'batavg': .240, 'obp': .280, 'slg': .350, 'slgmod': .355, 'obs': .630, 'sobat': .2}
dd_rest = {'batavg': .255, 'obp': .310, 'slg': .380, 'slgmod': .430, 'obs': .690, 'sobat': .2}
dd['p'] = dd_p
dd['ss'] = dd_ss_c
dd['c'] = dd_ss_c
dd['2b'] = dd_2b_3b
dd['3b'] = dd_2b_3b
dd['1b'] = dd_rest
dd['lf'] = dd_rest
dd['rf'] = dd_rest
dd['cf'] = dd_rest
dd['ph'] = dd_rest
dd['pr'] = dd_ss_c
dd['dh'] = dd_rest


In [5]:
def process_batter_df(p_id, dict_def):
    fname = 'data/batter_data/batting_data_'+p_id+'.csv'
    try:
        batter_df = pd.read_csv(fname)
        pos = batter_df.Pos.mode()[0]
        
        # corner cases where the most common position was a pair
        if ',' in pos:
            pos = pos.split(',')[0]
            
        batter_df['Date'] = (pd.to_datetime(batter_df.Date).astype(str).str.replace('-','')).astype(int)
        t_col = batter_df['dblhead_num'].copy()
        t_col[np.isnan(t_col)] = 0
        batter_df['dblheader_int'] = t_col.astype(int)
        for winsize in [30,75,162,350]:
            suff = str(winsize)
            for raw_col in ['AB','BB','H','2B','3B','HR','HBP','SO','SB','CS']:
                new_col = 'rollsum_'+raw_col+'_'+suff
                batter_df[new_col] = roll_column(batter_df, raw_col, winsize)

            ab_per_game_def = 2
            pa_per_game_def = 2
            batavg_def = dict_def[pos]['batavg']
            obp_def = dict_def[pos]['obp']
            slg_def = dict_def[pos]['slg']
            slgmod_def = dict_def[pos]['slgmod']
            so_bat_perc_def = dict_def[pos]['sobat']

            # Columns created by aggregation above
            ab_col = 'rollsum_AB_'+str(winsize)
            h_col = 'rollsum_H_'+str(winsize)
            bb_col = 'rollsum_BB_'+str(winsize)
            hbp_col = 'rollsum_HBP_'+str(winsize)
            doub_col = 'rollsum_2B_'+str(winsize)
            trip_col = 'rollsum_3B_'+str(winsize)
            hr_col = 'rollsum_HR_'+str(winsize)
            so_col = 'rollsum_SO_'+str(winsize)

            # Columns I will define below
            abmod_col = 'ABmod_'+str(winsize)
            fakeab_col = 'fakeAB_'+str(winsize)
            pa_col = 'PA_'+str(winsize)
            pamod_col = 'PAmod_'+str(winsize)
            fakepa_col = 'fakePA_'+str(winsize)
            xb_col = 'XB_'+str(winsize) # represents extra bases beyond hits
            slg_col = 'SLG_'+str(winsize)
            slgmod_col = 'SLGmod_'+str(winsize)
            batavg_col = 'BATAVG_'+str(winsize)
            so_bat_perc_col = 'SObat_perc_'+str(winsize)
            obp_col = 'OBP_'+str(winsize)
            obs_col = 'OBS_'+str(winsize)

            # calculate BATAVG, with smoothing for low AB numbers
            batter_df[abmod_col] = np.maximum(batter_df[ab_col],winsize*ab_per_game_def)
            batter_df[fakeab_col] = np.minimum(batter_df[abmod_col]-batter_df[ab_col],0)
            batter_df[batavg_col] = (batter_df[h_col] + (batter_df[fakeab_col]*batavg_def))/(batter_df[abmod_col])

            # calculate SLG, with smoothing for low AB numbers
            batter_df[xb_col] = batter_df[doub_col] + 2*batter_df[trip_col] + 3*batter_df[hr_col]
            batter_df[slg_col] = (batter_df[h_col] + batter_df[xb_col] + 
                                     (batter_df[fakeab_col]*slg_def))/(batter_df[abmod_col])

            # calculate OBP, with smoothing for low PA numbers
            batter_df[pa_col] = batter_df[ab_col]+batter_df[bb_col]+batter_df[hbp_col]
            batter_df[pamod_col] = np.maximum(batter_df[pa_col],winsize*pa_per_game_def)
            batter_df[fakepa_col] = np.minimum(batter_df[pamod_col]-batter_df[pa_col],0)
            batter_df[obp_col] = (batter_df[h_col] + batter_df[bb_col] + batter_df[hbp_col]
                                  + (batter_df[fakepa_col]*obp_def))/(
                                    batter_df[pamod_col])

            # calculate SLGmod, with smoothing for low PA numbers
            batter_df[slgmod_col] = (batter_df[so_col] + batter_df[bb_col] + batter_df[hbp_col] 
                                     +batter_df[xb_col] + (batter_df[fakepa_col]*slgmod_def))/(
                                    batter_df[pamod_col])

            # calculate SObat_perc, with smoothing for low PA numbers
            batter_df[so_bat_perc_col] = (batter_df[so_col] + (batter_df[fakepa_col]*so_bat_perc_def))/(
                                    batter_df[pamod_col])

            # calculate OBS
            batter_df[obs_col] = batter_df[obp_col]+batter_df[slg_col]


            batter_df['date_dblhead'] = (batter_df['Date'].astype(str) + batter_df['dblheader_int'].astype(str)).astype(int)
            batter_df.set_index('date_dblhead', inplace=True)
    except:
        try:
            print(f'issue for {fname} at position {pos}, returning None')
        except:
            print(f'issue for {fname}, returning None')
        batter_df = None
    return(batter_df)

In [6]:
batter_ids = np.array([])
for num in range(1,10):
    for suffix in ['_h','_v']:
        # Check whether this should be '_id' or '_name'
        colname = 'batter'+str(num)+'_name'+suffix
        batter_ids = np.concatenate((batter_ids, pd.unique(df[colname])))
batter_ids = np.unique(batter_ids)

In [7]:
## Create a batter data dictionary from the saved batter files
batter_data_dict = {}
for i,b_id in enumerate(batter_ids):
    if i%100==0:
        print(i)
    batter_data_dict[b_id] = process_batter_df(b_id, dd)

0
100
200
issue for data/batter_data/batting_data_baezm001.csv, returning None
300
issue for data/batter_data/batting_data_bedrc001.csv, returning None
400
500
600
issue for data/batter_data/batting_data_boshb001.csv, returning None
issue for data/batter_data/batting_data_brogc001.csv, returning None
700
800
900
issue for data/batter_data/batting_data_castm002.csv, returning None
1000
issue for data/batter_data/batting_data_claua001.csv, returning None
1100
issue for data/batter_data/batting_data_colet001.csv, returning None
issue for data/batter_data/batting_data_contr001.csv, returning None
1200
1300
1400
1500
1600
1700
1800
1900
issue for data/batter_data/batting_data_garcf003.csv, returning None
2000
issue for data/batter_data/batting_data_gelts001.csv, returning None
2100
issue for data/batter_data/batting_data_gonzv001.csv, returning None
2200
issue for data/batter_data/batting_data_greec004.csv, returning None
2300
2400
2500
2600
2700
2800
2900
issue for data/batter_data/batting

In [8]:
new_col_dict = {}
colstems = ['BATAVG', 'OBP', 'SLG', 'OBS', 'SLGmod','SObat_perc']
winsizes = [30,75,162,350]
new_col_list = [stem+'_'+str(winsize)+'_b'+str(i)+hv   for stem in colstems for winsize in winsizes 
                for i in range(1,10) for hv in ['_h','_v']]
for col in new_col_list:
    new_col_dict[col] = np.empty(df.shape[0])
    new_col_dict[col].fill(np.nan)

In [9]:
def get_batter_ids_from_row(row):
    b_cols = ['batter1_name_h', 'batter1_name_v', 'batter2_name_h',
       'batter2_name_v', 'batter3_name_h', 'batter3_name_v',
       'batter4_name_h', 'batter4_name_v', 'batter5_name_h',
       'batter5_name_v', 'batter6_name_h', 'batter6_name_v',
       'batter7_name_h', 'batter7_name_v', 'batter8_name_h',
       'batter8_name_v', 'batter9_name_h', 'batter9_name_v']
    return(row.loc[b_cols].to_dict())

In [10]:
## Add in all the statistics about the starting lineup batters
for i in range(df.shape[0]):
    row = df.iloc[i,:]
    if i%1000==0:
        print(i)
    bid_dict = get_batter_ids_from_row(row)
    date_dblhead = row['date_dblhead']
    for hv in ['_h','_v']:
        for j in range(1,10):
            # Again, check if should be name or id col
            #curr_col = 'batter'+str(j)+'_id'+hv
            curr_col = 'batter'+str(j)+'_name'+hv
            curr_b_id = bid_dict[curr_col]
            if curr_b_id in batter_data_dict.keys():
                curr_batter_df = batter_data_dict[curr_b_id]
                if (curr_batter_df is not None) and (curr_batter_df.shape[0]>0):
                    try:
                        curr_batter_row = curr_batter_df.loc[date_dblhead,:]
                    except:
                        print(f'date not found for batter {curr_b_id} game {date_dblhead}')
                        prev_game_indices = np.where(curr_batter_df.index<date_dblhead)[0]
                        if len(prev_game_indices)==0:
                            index_to_use = 0
                        else:
                            index_to_use = np.max(prev_game_indices)
                        curr_batter_row = curr_batter_df.iloc[index_to_use,:]
                        print(f'using date {curr_batter_df.index[index_to_use]}')
                    if (curr_batter_row.ndim>1):
                        curr_batter_row = curr_batter_row.iloc[0,:]
                    for stem in colstems:
                        for winsize in winsizes:
                            newcolname = stem+'_'+str(winsize)+'_b'+str(j)+hv
                            new_col_dict[newcolname][i] = curr_batter_row[stem+'_'+str(winsize)]
                else:
                    print(f'No data found for {curr_b_id}')
            else:
                print(f'batter not found for {curr_b_id}')

0
1000
2000
3000
4000
5000
6000
7000
8000
date not found for batter schud001 game 198405260
using date 198310010
9000
10000
No data found for oconj001
11000
12000
13000
date not found for batter smitm001 game 198609200
using date 198404220
14000
date not found for batter penah001 game 198610050
using date 198704290
15000
date not found for batter lea-c001 game 198709160
using date 198409160
16000
date not found for batter mallr001 game 198710020
using date 199108170
17000
date not found for batter krueb001 game 198807040
using date 199209050
18000
19000
20000
21000
22000
23000
24000
25000
26000
No data found for vitkj001
27000
28000
29000
30000
31000
32000
date not found for batter barbb002 game 199604140
using date 199510010
33000
No data found for quirr001
34000
date not found for batter cormr001 game 199704050
using date 199609270
35000
36000
No data found for karpr001
37000
date not found for batter myerr002 game 199709270
using date 199609272
38000
39000
date not found for batter 

In [11]:
for key, val in new_col_dict.items():
    df[key] = val

/var/folders/q8/xr46tkk15h10554_3mptdjlw0000gn/T/ipykernel_22031/606030580.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key] = val


In [12]:
df = df.copy()

In [13]:
newcols = new_col_dict.keys()

## Deal with missing data

In [14]:
df.loc[:,newcols].isnull().sum()

BATAVG_30_b1_h          0
BATAVG_30_b1_v          0
BATAVG_30_b2_h          0
BATAVG_30_b2_v          0
BATAVG_30_b3_h          0
BATAVG_30_b3_v          0
BATAVG_30_b4_h          0
BATAVG_30_b4_v          0
BATAVG_30_b5_h          0
BATAVG_30_b5_v          0
BATAVG_30_b6_h          0
BATAVG_30_b6_v          0
BATAVG_30_b7_h          0
BATAVG_30_b7_v          0
BATAVG_30_b8_h          0
BATAVG_30_b8_v          1
BATAVG_30_b9_h         10
BATAVG_30_b9_v         22
BATAVG_75_b1_h          0
BATAVG_75_b1_v          0
BATAVG_75_b2_h          0
BATAVG_75_b2_v          0
BATAVG_75_b3_h          0
BATAVG_75_b3_v          0
BATAVG_75_b4_h          0
BATAVG_75_b4_v          0
BATAVG_75_b5_h          0
BATAVG_75_b5_v          0
BATAVG_75_b6_h          0
BATAVG_75_b6_v          0
BATAVG_75_b7_h          0
BATAVG_75_b7_v          0
BATAVG_75_b8_h          0
BATAVG_75_b8_v          1
BATAVG_75_b9_h         10
BATAVG_75_b9_v         22
BATAVG_162_b1_h         0
BATAVG_162_b1_v         0
BATAVG_162_b

In [15]:
newcols89 = [stem+'_'+str(winsize)+'_b'+str(i)+hv   for stem in colstems for winsize in winsizes 
                 for hv in ['_h','_v'] for i in range(8,10)]
for col in newcols89:
    stem=col.split('_')[0].lower()
    df[col].fillna(dd['p'][stem])

### Create some summary features

In [16]:
w9 = np.array([0.12541131, 0.12159052, 0.11787189, 0.11434144, 0.11096691,
       0.10772781, 0.10430724, 0.10078822, 0.09699465])
w8 = w9[:-1]/np.sum(w9[:-1])

In [17]:
for col in ['BATAVG', 'OBP', 'SLG', 'OBS', 'SLGmod','SObat_perc']:
    for winsize in winsizes:
        for hv in ['_h','_v']:
            b_cols9 = [col+'_'+str(winsize)+'_b'+str(i)+hv for i in range(1,10)]
            b_cols8 = [col+'_'+str(winsize)+'_b'+str(i)+hv for i in range(1,9)]
            fcolname9 = 'lineup9_'+col+'_'+str(winsize)+hv
            fcolname8 = 'lineup8_'+col+'_'+str(winsize)+hv
            fcolname9w = 'lineup9_'+col+'_'+str(winsize)+'_w'+hv
            fcolname8w = 'lineup8_'+col+'_'+str(winsize)+'_w'+hv
            df[fcolname9] = np.mean(df.loc[:,b_cols9].to_numpy(),axis=1)
            df[fcolname8] = np.mean(df.loc[:,b_cols8].to_numpy(),axis=1)
            df[fcolname9w] = df.loc[:,b_cols9].to_numpy().dot(w9)
            df[fcolname8w] = df.loc[:,b_cols8].to_numpy().dot(w8)

In [18]:
df.to_csv('df_bp9.csv', index=False)